In [16]:
import sklearn
import time
import pandas as pd   
from matplotlib import pyplot
from numpy import mean
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, RepeatedKFold, GridSearchCV, cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB, CategoricalNB
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier


In [2]:
def robust_scale(df):
    numerical_columns = df.select_dtypes(include=['number']).columns
    # df.drop(columns=['RecordID','hospital_id'], inplace=True)
    scaler = RobustScaler()
    df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

    return df

def knn_impute_numerical_columns(df, n_neighbors=10):
    df_imputed = df.copy()
    
    categorical_columns = df_imputed.select_dtypes(exclude=['number']).columns
    numerical_cols = df_imputed.select_dtypes(include=['number']).columns
    
    imputer = KNNImputer(n_neighbors=n_neighbors)
    # imputer = SimpleImputer(strategy='mean') 
    df_imputed[numerical_cols] = imputer.fit_transform(df_imputed[numerical_cols])
    
    # Label encode categorical columns
    # label_encoders = {}
    # for col in categorical_columns:
    #     label_encoders[col] = LabelEncoder()
    #     df_imputed[col] = label_encoders[col].fit_transform(df_imputed[col])

    # imputer = SimpleImputer(strategy='constant', fill_value='Unknown')
    imputer = SimpleImputer(strategy='most_frequent')
    df_imputed[categorical_columns] = imputer.fit_transform(df_imputed[categorical_columns])
    
    # cat_imputer = SimpleImputer(strategy='most_frequent')
    # df_imputed[categorical_columns] = cat_imputer.fit_transform(df_imputed[categorical_columns])

    return df_imputed


In [3]:
df = pd.read_csv('train.csv')



In [4]:
robust_scale(df)

,RecordID,hospital_id,icu_id,ethnicity,gender,icu_admit_source,icu_stay_type,icu_type,apache_3j_bodysystem,apache_2_bodysystem,...,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,d1_glucose_max,d1_potassium_max,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,immunosuppression,solid_tumor_with_metastasis,hospital_death
0,-1.00000,-0.666667,1.241270,Caucasian,M,Floor,transfer,Med-Surg ICU,Metabolic,Metabolic,...,-0.805556,-1.000000,-0.805556,NaN,NaN,-0.333333,-0.333333,0.0,0.0,0.0
1,-0.99996,-0.789474,0.012698,African American,M,Accident & Emergency,admit,Med-Surg ICU,Cardiovascular,Cardiovascular,...,-0.555556,-0.945946,-0.555556,2.164706,-0.500,NaN,NaN,0.0,0.0,0.0
2,-0.99992,-0.429825,-0.073016,Caucasian,M,Floor,admit,MICU,Respiratory,Respiratory,...,1.305556,1.189189,1.305556,0.129412,0.125,2.750000,2.166667,0.0,0.0,0.0
3,-0.99988,-0.815789,0.860317,Caucasian,M,Accident & Emergency,admit,Med-Surg ICU,Neurological,Neurologic,...,0.694444,0.891892,0.694444,1.282353,-0.375,0.583333,0.666667,0.0,0.0,0.0
4,-0.99984,0.745614,0.968254,Caucasian,F,Accident & Emergency,admit,CSICU,Cardiovascular,Cardiovascular,...,0.111111,-0.297297,0.111111,-0.458824,-0.250,0.833333,0.833333,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0.99984,-0.807018,0.473016,Caucasian,M,Accident & Emergency,admit,Med-Surg ICU,Musculoskeletal/Skin,Undefined Diagnoses,...,-0.472222,-0.864865,-0.472222,-0.600000,-0.250,-0.250000,-0.166667,0.0,0.0,0.0
49996,0.99988,-0.412281,0.568254,Caucasian,F,Floor,admit,Med-Surg ICU,Neurological,Neurologic,...,0.833333,0.405405,0.833333,-0.047059,-0.375,-0.250000,-0.333333,0.0,0.0,0.0
49997,0.99992,-0.956140,1.180952,Caucasian,F,Accident & Emergency,transfer,Med-Surg ICU,Sepsis,Cardiovascular,...,-0.833333,-0.405405,-0.833333,-0.376471,NaN,0.333333,0.500000,1.0,0.0,0.0
49998,0.99996,0.245614,0.571429,Caucasian,M,Operating Room / Recovery,admit,Cardiac ICU,Cardiovascular,Cardiovascular,...,0.611111,0.189189,0.611111,-0.352941,-0.125,-0.416667,-0.333333,0.0,0.0,0.0


In [5]:
df_imputed = knn_impute_numerical_columns(df, n_neighbors=50)

In [6]:
df_dropped = df_imputed.copy().dropna(axis=0)
df_dropped.shape
# missing_value_counts(df_dropped)

(50000, 58)

In [7]:
df_onehot = pd.get_dummies(df_dropped)
df_onehot
df_onehot.shape

(50000, 96)

In [8]:
def label_encode_categorical_columns(df):
    df_encoded = df.copy()
    label_encoder = LabelEncoder()
    categorical_cols = df_encoded.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        df_encoded[col] = label_encoder.fit_transform(df_encoded[col])
    return df_encoded

# Example usage:
# df_dropped = your_dataframe_here
# df_encoded = label_encode_categorical_columns(df_dropped)
# print(df_encoded)


In [130]:
X = df_onehot.loc[:, df_onehot.columns != "hospital_death"]
y = df_onehot[["hospital_death"]]


X.shape

(50000, 95)

In [10]:
from sklearn.ensemble import RandomForestClassifier

estimator = RandomForestClassifier(n_estimators=1000, max_depth=7, random_state=0, n_jobs=3)
estimator.fit(X, y)

# Get feature importances
feature_importances = estimator.feature_importances_


feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
})

feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

feature_num = 75
selected_features = feature_importance_df['Feature'][:feature_num]
selected_features

/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


46            apache_4a_hospital_death_prob
47                 apache_4a_icu_death_prob
10                         gcs_motor_apache
24                              d1_spo2_min
9                           gcs_eyes_apache
                      ...                  
50               ethnicity_African American
75    apache_3j_bodysystem_Gastrointestinal
57                                 gender_M
63                      icu_stay_type_admit
62               icu_admit_source_Other ICU
Name: Feature, Length: 75, dtype: object

In [36]:
X = X[selected_features]

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X.shape

(50000, 75)

In [129]:
def roc_auc_cv(model, X, y):
    cv = RepeatedKFold(n_splits=10, n_repeats=2, random_state=1)
    predicted_probabilities = cross_val_score(model, X, y, scoring="roc_auc", cv=cv, n_jobs=2, verbose=2)

    return mean(predicted_probabilities)
def calculate_roc_auc(model, X_test, y_test):
    md_probs = model.predict_proba(X_test)
    md_probs = md_probs[:, 1]
    md_auc = roc_auc_score(y_test, md_probs)
    return md_auc

# roc_auc = calculate_roc_auc(model, X_test, y_test)
# print(roc_auc)


In [112]:
from catboost import CatBoostClassifier


In [127]:
X.shape

(50000, 75)

In [128]:

model3 = RandomForestClassifier(n_estimators=300, max_depth=10, random_state=0,
                               n_jobs=2, verbose=2, min_samples_leaf=5, max_features=0.2)

model1 = XGBClassifier(n_estimators=200, max_depth=5, random_state=0, n_jobs=3, learning_rate=0.1, colsample_bytree=0.13)
model2 = CatBoostClassifier(n_estimators=1000, max_depth=5, random_state=0, learning_rate=0.038, )
model = VotingClassifier(estimators=[('xgb', model1), ('cat', model2), ('rf', model3)], voting='soft', n_jobs=2)

# model.fit(X_train, y_train)
# roc_auc = calculate_roc_auc(model, X_test, y_test)
# print(roc_auc)
print(roc_auc_cv(model, X, y))


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/homebrew

0:	learn: 0.6451037	total: 74.5ms	remaining: 1m 14s
0:	learn: 0.6452176	total: 90.6ms	remaining: 1m 30s
1:	learn: 0.6046076	total: 95.5ms	remaining: 47.6s
1:	learn: 0.6045309	total: 109ms	remaining: 54.5s
0:	learn: 0.6451995	total: 76ms	remaining: 1m 15s
2:	learn: 0.5674272	total: 121ms	remaining: 40.1s
0:	learn: 0.6450990	total: 94.2ms	remaining: 1m 34s
1:	learn: 0.6045310	total: 93.2ms	remaining: 46.5s
3:	learn: 0.5329506	total: 141ms	remaining: 35.1s
2:	learn: 0.5671177	total: 147ms	remaining: 48.8s
2:	learn: 0.5671697	total: 108ms	remaining: 36s
4:	learn: 0.5013694	total: 158ms	remaining: 31.5s
1:	learn: 0.6045955	total: 125ms	remaining: 1m 2s
3:	learn: 0.5327268	total: 172ms	remaining: 42.8s
3:	learn: 0.5327152	total: 147ms	remaining: 36.6s
2:	learn: 0.5672770	total: 151ms	remaining: 50s
4:	learn: 0.5012528	total: 192ms	remaining: 38.1s
5:	learn: 0.4727543	total: 201ms	remaining: 33.3s
3:	learn: 0.5329233	total: 173ms	remaining: 43s
5:	learn: 0.4726165	total: 218ms	remaining: 36.2

[Parallel(n_jobs=2)]: Done  40 tasks      | elapsed:    5.7s
[Parallel(n_jobs=2)]: Done  40 tasks      | elapsed:    5.8s


302:	learn: 0.1897892	total: 10.3s	remaining: 23.7s
304:	learn: 0.1894257	total: 10.3s	remaining: 23.6s
305:	learn: 0.1893769	total: 10.4s	remaining: 23.5s
301:	learn: 0.1886868	total: 10.3s	remaining: 23.7s
303:	learn: 0.1897267	total: 10.4s	remaining: 23.7s
building tree 40 of 300
301:	learn: 0.1901335	total: 10.5s	remaining: 24.2s
306:	learn: 0.1893305	total: 10.4s	remaining: 23.5s
building tree 41 of 300
302:	learn: 0.1886383	total: 10.3s	remaining: 23.7s
304:	learn: 0.1896785	total: 10.4s	remaining: 23.7s
303:	learn: 0.1885841	total: 10.3s	remaining: 23.6s
302:	learn: 0.1900780	total: 10.5s	remaining: 24.1s
307:	learn: 0.1892824	total: 10.4s	remaining: 23.4s
building tree 40 of 300
304:	learn: 0.1885330	total: 10.3s	remaining: 23.5s
303:	learn: 0.1900094	total: 10.5s	remaining: 24.1s
308:	learn: 0.1892410	total: 10.4s	remaining: 23.4s
305:	learn: 0.1896235	total: 10.4s	remaining: 23.6s
304:	learn: 0.1899516	total: 10.5s	remaining: 24s
305:	learn: 0.1884868	total: 10.4s	remaining: 

[Parallel(n_jobs=2)]: Done  40 tasks      | elapsed:    5.9s
[Parallel(n_jobs=2)]: Done  40 tasks      | elapsed:    5.7s


...

building tree 293 of 300
building tree 297 of 300
building tree 294 of 300
[CV] END .................................................... total time=  45.8s
building tree 298 of 300
building tree 295 of 300
building tree 299 of 300
building tree 296 of 300
building tree 300 of 300
building tree 297 of 300
building tree 298 of 300


/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:312: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:314: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:345: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:336: FutureWarning: is_categorical_dtype is deprecated and will be removed in a f

building tree 299 of 300
[CV] END .................................................... total time=  45.7s
building tree 300 of 300


/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:312: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:314: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:345: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:336: FutureWarning: is_categorical_dtype is deprecated and will be removed in a f

[CV] END .................................................... total time=  45.7s
0.8844239133353954


In [32]:

clf1 = DecisionTreeClassifier(random_state=42, max_depth=7)
clf2 = RandomForestClassifier(random_state=42, n_estimators=700, max_depth=6, n_jobs=3, verbose=2)
clf3 = GradientBoostingClassifier(random_state=42, n_estimators=500, max_depth=2, learning_rate=0.2, verbose=2)
clf5 = XGBClassifier(random_state=42, n_estimators=800, max_depth=4, learning_rate=0.038)
clf6 = CatBoostClassifier(random_state=42, n_estimators=590, max_depth=5, learning_rate=0.09, l2_leaf_reg=2, border_count=80, leaf_estimation_iterations=10)

voting_clf = VotingClassifier(estimators=[('gb', clf3), ('xgb', clf5), ('cat', clf6)], voting='soft', n_jobs=3, verbose=2)

# Train the Voting Classifier
voting_clf.fit(X_train, y_train)

# Make predictions using the Voting Classifier
y_pred = voting_clf.predict(X_test)

calculate_roc_auc(voting_clf, X_test, y_test)


/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:312: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:314: FutureWarn

      Iter       Train Loss   Remaining Time 
         1           0.5167            1.25m
         2           0.4904            1.19m
         3           0.4736            1.14m
         4           0.4611            1.11m
         5           0.4527            1.09m
         6           0.4463            1.09m
0:	learn: 0.5908458	total: 66.3ms	remaining: 39s
1:	learn: 0.5028324	total: 75ms	remaining: 22s
2:	learn: 0.4344424	total: 83.6ms	remaining: 16.4s
3:	learn: 0.3819253	total: 96.3ms	remaining: 14.1s
4:	learn: 0.3478244	total: 108ms	remaining: 12.6s
5:	learn: 0.3223018	total: 116ms	remaining: 11.3s
         7           0.4431            1.08m
6:	learn: 0.3007924	total: 125ms	remaining: 10.4s
7:	learn: 0.2849698	total: 133ms	remaining: 9.71s
8:	learn: 0.2716279	total: 143ms	remaining: 9.23s
9:	learn: 0.2597113	total: 153ms	remaining: 8.85s
10:	learn: 0.2511295	total: 161ms	remaining: 8.49s
11:	learn: 0.2426372	total: 170ms	remaining: 8.17s
12:	learn: 0.2362720	total: 178ms	remai

/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:312: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:314: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:345: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:336: FutureWarning: is_categorical_dtype is deprecated and will be removed in a f

0.8867887123602034

In [27]:
model = voting_clf

In [35]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingRegressor
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# Define your base models
base_models = [
    ('xgb', XGBClassifier(n_estimators=800, random_state=42, learning_rate= 0.038, max_depth=4)),
    ('cb', CatBoostClassifier(n_estimators=480, random_state=42, learning_rate= 0.092, max_depth=5)),
]

# Define your meta-model (XGBoost)
meta_model = CatBoostClassifier(n_estimators=590, random_state=42, learning_rate= 0.09, max_depth=5, l2_leaf_reg=2, border_count=80, leaf_estimation_iterations=10)

# Create a stacking model
stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_model)

# Train the stacking model
stacking_model.fit(X_train, y_train)

# Make predictions using the stacking model
y_pred = stacking_model.predict(X_test)

model = stacking_model
print(calculate_roc_auc(model, X_test, y_test))

/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:312: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:314: FutureWarn

0:	learn: 0.5863486	total: 10.6ms	remaining: 5.08s
1:	learn: 0.4952395	total: 17ms	remaining: 4.06s
2:	learn: 0.4327245	total: 23.9ms	remaining: 3.8s
3:	learn: 0.3869539	total: 30.9ms	remaining: 3.68s
4:	learn: 0.3493746	total: 37.4ms	remaining: 3.55s
5:	learn: 0.3247264	total: 44.9ms	remaining: 3.55s
6:	learn: 0.3028286	total: 51.2ms	remaining: 3.46s
7:	learn: 0.2830368	total: 57.7ms	remaining: 3.4s
8:	learn: 0.2697172	total: 64.1ms	remaining: 3.35s
9:	learn: 0.2583589	total: 70.5ms	remaining: 3.31s
10:	learn: 0.2487955	total: 76.8ms	remaining: 3.28s
11:	learn: 0.2412544	total: 83.4ms	remaining: 3.25s
12:	learn: 0.2347629	total: 90ms	remaining: 3.23s
13:	learn: 0.2289109	total: 97ms	remaining: 3.23s
14:	learn: 0.2257740	total: 104ms	remaining: 3.21s
15:	learn: 0.2227317	total: 110ms	remaining: 3.19s
16:	learn: 0.2206200	total: 117ms	remaining: 3.18s
17:	learn: 0.2179563	total: 124ms	remaining: 3.17s
18:	learn: 0.2163521	total: 131ms	remaining: 3.17s
19:	learn: 0.2147106	total: 137ms	r

/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:312: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:314: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:345: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:336: FutureWarning: is_categorical_dtype is deprecated and will be removed in a f

0:	learn: 0.5856044	total: 6.98ms	remaining: 3.34s
1:	learn: 0.4945327	total: 12.6ms	remaining: 3.01s
2:	learn: 0.4319331	total: 16.9ms	remaining: 2.69s
3:	learn: 0.3868197	total: 22ms	remaining: 2.62s
4:	learn: 0.3492506	total: 27.5ms	remaining: 2.62s
5:	learn: 0.3230099	total: 33.6ms	remaining: 2.65s
6:	learn: 0.3012873	total: 39.1ms	remaining: 2.65s
7:	learn: 0.2816850	total: 45.6ms	remaining: 2.69s
8:	learn: 0.2661196	total: 51.3ms	remaining: 2.68s
9:	learn: 0.2552504	total: 57.6ms	remaining: 2.71s
10:	learn: 0.2485413	total: 63.3ms	remaining: 2.7s
11:	learn: 0.2408194	total: 69.6ms	remaining: 2.71s
12:	learn: 0.2342240	total: 75.6ms	remaining: 2.71s
13:	learn: 0.2289538	total: 81.3ms	remaining: 2.71s
14:	learn: 0.2259422	total: 87.3ms	remaining: 2.71s
15:	learn: 0.2218751	total: 94ms	remaining: 2.73s
16:	learn: 0.2197936	total: 99.7ms	remaining: 2.71s
17:	learn: 0.2169131	total: 106ms	remaining: 2.71s
18:	learn: 0.2151826	total: 112ms	remaining: 2.71s
19:	learn: 0.2136605	total: 1

/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:312: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:314: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:345: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:336: FutureWarning: is_categorical_dtype is deprecated and will be removed in a f

0.877971641689071


In [ ]:
df_catboost = pd.read_csv('train.csv')
df_catboost = pd.get_dummies(df_catboost)
X = df_catboost.loc[:, df_catboost.columns != "hospital_death"]
y = df_catboost[["hospital_death"]]
# X = X[selected_features]


In [ ]:
X.shape 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
from catboost import CatBoostClassifier, Pool


model = CatBoostClassifier(
    iterations=590,
    depth=5,
    learning_rate=0.09,
    loss_function='Logloss',
    eval_metric='AUC',
    random_seed=69,
    verbose=20,
    l2_leaf_reg=2,
    border_count=80,
    leaf_estimation_iterations=10,
    # subsample=0.8,
    # min_data_in_leaf=3,
    # colsample_bylevel=0.8,
    
)
# roc_auc_cv(model, X, y)
model.fit(X_train, y_train)
calculate_roc_auc(model, X_test, y_test)

In [ ]:
from xgboost import XGBClassifier

model = XGBClassifier(
    n_estimators=800,  
    max_depth=4,
    learning_rate=0.038,
    random_state=49,
    min_child_weight=1,
    colsample_bytree=0.78,
    alpha= 0.1,
    eval_metric='logloss',
)
# roc_auc_cv(model, X, y)

# model.fit(X_train, y_train)
roc_auc_cv(model, X, y)
# calculate_roc_auc(model, X_test, y_test)
# print(roc)
# model.get_params()

In [122]:
model.fit(X, y)
roc_auc_cv(model, X, y)

/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:312: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:314: FutureWarn

0:	learn: 0.6453279	total: 63ms	remaining: 1m 2s
1:	learn: 0.6048206	total: 71.9ms	remaining: 35.9s
2:	learn: 0.5676233	total: 79.6ms	remaining: 26.5s
3:	learn: 0.5334325	total: 89.1ms	remaining: 22.2s
4:	learn: 0.5022437	total: 97.6ms	remaining: 19.4s
5:	learn: 0.4732019	total: 107ms	remaining: 17.8s
6:	learn: 0.4499869	total: 116ms	remaining: 16.5s
7:	learn: 0.4292836	total: 124ms	remaining: 15.4s
8:	learn: 0.4081345	total: 132ms	remaining: 14.6s
9:	learn: 0.3893917	total: 142ms	remaining: 14.1s
10:	learn: 0.3730228	total: 151ms	remaining: 13.6s
11:	learn: 0.3565125	total: 160ms	remaining: 13.2s
12:	learn: 0.3415375	total: 172ms	remaining: 13.1s
13:	learn: 0.3291922	total: 180ms	remaining: 12.7s
14:	learn: 0.3174116	total: 190ms	remaining: 12.5s
15:	learn: 0.3075112	total: 199ms	remaining: 12.2s
16:	learn: 0.2995256	total: 207ms	remaining: 12s
17:	learn: 0.2924996	total: 217ms	remaining: 11.8s
18:	learn: 0.2852507	total: 228ms	remaining: 11.8s
19:	learn: 0.2789342	total: 239ms	remain

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dty

0:	learn: 0.6450990	total: 32.1ms	remaining: 32.1s
0:	learn: 0.6451995	total: 28.5ms	remaining: 28.4s
1:	learn: 0.6045955	total: 60.3ms	remaining: 30.1s
1:	learn: 0.6045310	total: 44.9ms	remaining: 22.4s
2:	learn: 0.5672770	total: 84.2ms	remaining: 28s
2:	learn: 0.5671697	total: 64.9ms	remaining: 21.6s
3:	learn: 0.5329233	total: 104ms	remaining: 25.9s
3:	learn: 0.5327152	total: 80.6ms	remaining: 20.1s
4:	learn: 0.5015043	total: 128ms	remaining: 25.4s
4:	learn: 0.5012088	total: 124ms	remaining: 24.8s
5:	learn: 0.4729103	total: 157ms	remaining: 26.1s
6:	learn: 0.4496754	total: 234ms	remaining: 33.2s
5:	learn: 0.4725211	total: 218ms	remaining: 36.2s
7:	learn: 0.4289226	total: 262ms	remaining: 32.4s
6:	learn: 0.4495466	total: 249ms	remaining: 35.3s
8:	learn: 0.4071125	total: 297ms	remaining: 32.7s
7:	learn: 0.4286424	total: 273ms	remaining: 33.9s
0:	learn: 0.6452176	total: 92.9ms	remaining: 1m 32s
1:	learn: 0.6045309	total: 125ms	remaining: 1m 2s
9:	learn: 0.3888033	total: 353ms	remaining:

/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:312: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:314: FutureWarn

...

0:	learn: 0.6456386	total: 38.5ms	remaining: 38.4s
1:	learn: 0.6020123	total: 67.3ms	remaining: 33.6s
2:	learn: 0.5650084	total: 119ms	remaining: 39.4s
3:	learn: 0.5311237	total: 200ms	remaining: 49.7s
4:	learn: 0.4986341	total: 227ms	remaining: 45.1s
5:	learn: 0.4702851	total: 262ms	remaining: 43.5s
6:	learn: 0.4475886	total: 309ms	remaining: 43.9s
7:	learn: 0.4268875	total: 343ms	remaining: 42.6s
0:	learn: 0.6453511	total: 58.8ms	remaining: 58.8s
0:	learn: 0.6454185	total: 122ms	remaining: 2m 1s
1:	learn: 0.6018376	total: 178ms	remaining: 1m 28s
1:	learn: 0.6018783	total: 137ms	remaining: 1m 8s
2:	learn: 0.5648570	total: 172ms	remaining: 57.3s
8:	learn: 0.4084103	total: 527ms	remaining: 58s
0:	learn: 0.6452745	total: 28.3ms	remaining: 28.3s
2:	learn: 0.5648947	total: 263ms	remaining: 1m 27s
3:	learn: 0.5306981	total: 210ms	remaining: 52.4s
1:	learn: 0.6047221	total: 76.8ms	remaining: 38.3s
9:	learn: 0.3898384	total: 610ms	remaining: 1m
3:	learn: 0.5307968	total: 350ms	remaining: 1m 2

/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:312: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:314: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:345: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:336: FutureWarning: is_categorical_dtype is deprecated and will be removed in a f

999:	learn: 0.1610764	total: 20.7s	remaining: 0us
[Voting] ...................... (2 of 2) Processing cat, total=  21.8s
[CV] END .................................................... total time=  21.7s
995:	learn: 0.1610777	total: 20.6s	remaining: 82.6ms
996:	learn: 0.1610346	total: 20.6s	remaining: 61.9ms
997:	learn: 0.1609888	total: 20.6s	remaining: 41.2ms
998:	learn: 0.1609596	total: 20.6s	remaining: 20.6ms
999:	learn: 0.1609147	total: 20.6s	remaining: 0us
[Voting] ...................... (2 of 2) Processing cat, total=  21.7s
[CV] END .................................................... total time=  21.8s
[CV] END .................................................... total time=  21.8s


/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:312: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:314: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:345: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:336: FutureWarning: is_categorical_dtype is deprecated and will be removed in a f

0.8851048604871113

In [29]:
X.shape

(50000, 95)

In [120]:
def generate_predictions_for_model(model, test_file, output_file):
    
    df = pd.read_csv(test_file)
    record_ids = df["RecordID"]
    
    robust_scale(df)
    df = knn_impute_numerical_columns(df, n_neighbors=50)
    # df = df.dropna(axis=0)
    df = pd.get_dummies(df)
    # df = label_encode_categorical_columns(df)
    
    X_test = df[selected_features] 
    # X_test = df.loc[:, df.columns != "hospital_death"]
    
    
    probs = model.predict_proba(X_test)
    probs = probs[:, 1]
    
    # Create a DataFrame for the results
    result = pd.DataFrame({'RecordID': record_ids, 'hospital_death': probs})
    
    # Save the results to a CSV file
    result.to_csv(output_file, index=False, header=["RecordID", "hospital_death"])


In [126]:
generate_predictions_for_model(model, "test.csv", "results89.csv")


/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:312: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:314: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:345: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/xgboost/data.py:336: FutureWarning: is_categorical_dtype is deprecated and will be removed in a f

In [ ]:
import lightgbm as lgb

train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

params = {
    'objective': 'binary',  
    'metric': 'binary_logloss', 
    'boosting_type': 'gbdt',  
    'num_leaves': 31,  
    'learning_rate': 0.05,
    'feature_fraction': 0.75,  
    'bagging_fraction': 0.90,  
    'bagging_freq': 5,  # 
    'verbose': 1,
    'max_bin': 255,
    'min_data_in_leaf': 40,  # Example value, adjust as needed
    'min_sum_hessian_in_leaf': 1e-4  ,# Example value, adjust as needed


}

num_round = 211# Number of boosting iterations
lightgbm_model = lgb.train(params, train_data, num_round, valid_sets=[test_data])

y_pred_prob = lightgbm_model.predict(X_test)

roc_auc = roc_auc_score(y_test, y_pred_prob)
print("ROC AUC:", roc_auc)
# roc_auc_cv(lightgbm_model, X, y)



In [ ]:
params = {
    'objective': 'binary',  # Binary classification
    'metric': 'binary_logloss',  # Logarithmic loss for binary classification
    'boosting_type': 'gbdt',  # Gradient boosting decision tree
    'num_leaves': 31,  # Maximum number of leaves in each tree
    'learning_rate': 0.05,  # Step size shrinkage
    'feature_fraction': 0.75,  # Fraction of features to be used in each iteration
    'bagging_fraction': 0.90,  # Fraction of data to be randomly sampled for training
    'bagging_freq': 5,  # Frequency for bagging
    'verbose': 0  # Verbosity level,    
}

num_round = 210  # Number of boosting iterations
lightgbm_model = lgb.train(params, train_data, num_round, valid_sets=[test_data])

y_pred_prob = lightgbm_model.predict(X_test)

roc_auc = roc_auc_score(y_test, y_pred_prob)
print("ROC AUC:", roc_auc)






In [ ]:
def gen_resultfile_lbgm(model, test_file, output_file):
    
    df_test = pd.read_csv(test_file)
    record_ids = df_test["RecordID"]
    
    df_scaled = robust_scale(df_test)
    df_test_imputed = knn_impute_numerical_columns(df_scaled, n_neighbors=100)
    df_test_onehot = pd.get_dummies(df_test_imputed)
    
    X_test = df_test_onehot[selected_features] 
    
    # X_test = df_test_onehot.loc[:, df_test_onehot.columns != "hospital_death"]
    
    probs = model.predict(X_test)
    
    # Create a DataFrame for the results
    result = pd.DataFrame({'RecordID': record_ids, 'hospital_death': probs})
    
    # Save the results to a CSV file
    result.to_csv(output_file, index=False, header=["RecordID", "hospital_death"])

In [ ]:
gen_resultfile_lbgm(lightgbm_model, "test.csv", "results54.csv")

In [ ]:
params = {
    'objective': 'binary',  # Binary classification
    'metric': 'binary_logloss',  # Logarithmic loss for binary classification
    'boosting_type': 'gbdt',  # Gradient boosting decision tree
    'num_leaves': 31,  # Maximum number of leaves in each tree
    'learning_rate': 0.05,  # Step size shrinkage
    'feature_fraction': 0.75,  # Fraction of features to be used in each iteration
    'bagging_fraction': 0.90,  # Fraction of data to be randomly sampled for training
    'bagging_freq': 5,  # Frequency for bagging
    'verbose': 0  # Verbosity level,    
}

num_round = 210  # Number of boosting iterations
lightgbm_model = lgb.train(params, train_data, num_round, valid_sets=[test_data])

y_pred_prob = lightgbm_model.predict(X_test)

roc_auc = roc_auc_score(y_test, y_pred_prob)
print("ROC AUC:", roc_auc)



# Make predictions using the trained model
record_ids = X["RecordID"]
probs = model.predict(X_test)

result = pd.DataFrame({'RecordID': record_ids, 'hospital_death': probs})
    
# Save the results to a CSV file
result.to_csv("results54", index=False, header=["RecordID", "hospital_death"])

In [ ]:
y_pred_prob = lightgbm_model.predict(X_test)

roc_auc = roc_auc_score(y_test, y_pred_prob)
print("ROC AUC:", roc_auc)


In [ ]:
X.shape

In [ ]:
train_data = pd.read_csv('train.csv')  # Replace with your training dataset path
test_data = pd.read_csv('test.csv')    # Replace with your other dataset path


# Create a LightGBM dataset from the training data
train_dataset = lgb.Dataset(X, label=y)

# Define LightGBM parameters
params = {
    'objective': 'binary',  # Binary classification
    'metric': 'binary_logloss',  # Logarithmic loss for binary classification
    'boosting_type': 'gbdt',  # Gradient boosting decision tree
    'num_leaves': 31,  # Maximum number of leaves in each tree
    'learning_rate': 0.05,  # Step size shrinkage
    'feature_fraction': 0.75,  # Fraction of features to be used in each iteration
    'bagging_fraction': 0.90,  # Fraction of data to be randomly sampled for training
    'bagging_freq': 5,  # Frequency for bagging
    'verbose': 0  # Verbosity level,    
}

num_boost_round = 210  # Number of boosting rounds

# Train the LightGBM model on the training dataset
model = lgb.train(params, train_dataset, num_boost_round=num_boost_round)

# Prepare the other dataset for predictions
X_test = test_data  # Preprocess your other dataset as needed

# Make predictions using the trained model
record_ids = X["RecordID"]
probs = lightgbm_model.predict(X_test)

result = pd.DataFrame({'RecordID': record_ids, 'hospital_death': probs})
    
# Save the results to a CSV file
result.to_csv("results54", index=False, header=["RecordID", "hospital_death"])



In [ ]:
test_df = pd.read_csv("test.csv")
robust_scale_df = robust_scale(test_df)
df_test_imputed = knn_impute_numerical_columns(robust_scale_df, n_neighbors=100)
df_test_onehot = pd.get_dummies(df_test_imputed)
X_test = df_test_onehot[selected_features]
X_test.shape

In [ ]:
y_pred_prob = lightgbm_model.predict(X_test)
y_pred_prob


In [ ]:
record_ids = test_df["RecordID"]
result = pd.DataFrame({'RecordID': record_ids, 'hospital_death': probs})
    
# Save the results to a CSV file
result.to_csv("results54.csv", index=False, header=["RecordID", "hospital_death"])